In [1]:
import os
try:
    from google.colab import drive
    COLAB = True
    print("Estoy en Google CoLab")
    %tensorflow_version 2.x
    !pip install livelossplot
except:
    os.environ["CUDA_VISIBLE_DEVICES"]="0" 
    print("No estoy en Google CoLab")
    COLAB = False

Estoy en Google CoLab
     |████████████████████████████████| 793 kB 7.1 MB/s 
     |████████████████████████████████| 381 kB 20.3 MB/s 
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.29 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.32.0 which is incompatible.


In [2]:
import tensorflow as tf
tf.__version__

'2.8.0'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
try:
  from livelossplot import PlotLossesKerasTF
except:
  !pip install livelossplot
  from livelossplot import PlotLossesKerasTF

from utils import draw2compare
from tensorflow.keras.layers import Layer

ModuleNotFoundError: ignored

### VAEs cuantificadas (VQ-VAE)

Vamos a necesitar una nueva capa para llevar a cabo la cuantización. 

In [ ]:
class VectorQuantizer(layers.Layer):
    def __init__(self, num_embeddings, embedding_dim, beta=0.25, **kwargs):
        super().__init__(**kwargs)
        self.embedding_dim = embedding_dim
        self.num_embeddings = num_embeddings
        self.beta = (
            beta  # Mantener entre [0.25, 2] según el autor.
        )

        # Inicialización del libro de códigos
        w_init = tf.random_uniform_initializer()
        self.embeddings = tf.Variable(
            initial_value=w_init(
                shape=(self.embedding_dim, self.num_embeddings), dtype="float32"
            ),
            trainable=True,
            name="embeddings_vqvae",
        )

    def call(self, x):
        # Tamaño de la entrada
        input_shape = tf.shape(x)
        
        # Llevamos a cabo el aplanado para generar los vectores continuos
        flattened = tf.reshape(x, [-1, self.embedding_dim])

        # Seleccionamos los índices y creamos vectores one-hot
        encoding_indices = self.get_code_indices(flattened)
        encodings = tf.one_hot(encoding_indices, self.num_embeddings)
        
        # Con los vectores one-hot y mediante un multiplicación generamos 
        # la vectorización a partir de libro de códigos
        quantized = tf.matmul(encodings, self.embeddings, transpose_b=True)
        quantized = tf.reshape(quantized, input_shape)

        # Calculamos los errores de alineamiento
        
        # (3r término)
        commitment_loss = self.beta * tf.reduce_mean(
            (tf.stop_gradient(quantized) - x) ** 2
        )
        
        # (2nd término)
        codebook_loss = tf.reduce_mean((quantized - tf.stop_gradient(x)) ** 2)
        self.add_loss(commitment_loss + codebook_loss)

        # devolvemos la condificación haciendo un bypass 
        quantized = x + tf.stop_gradient(quantized - x)
        return quantized

    def get_code_indices(self, flattened_inputs):
        # Usamos la distancia euclidea para calcular los índices
        similarity = tf.matmul(flattened_inputs, self.embeddings)
        distances = (
            tf.reduce_sum(flattened_inputs ** 2, axis=1, keepdims=True)
            + tf.reduce_sum(self.embeddings ** 2, axis=0)
            - 2 * similarity
        )

        # Derive the indices for minimum distances.
        encoding_indices = tf.argmin(distances, axis=1)
        return encoding_indices
    

# El artículo usa capas residuales en códificador y el decodificador.
class ResidualLayer(Layer):

    def __init__(self, kernels=128):
        super(ResidualLayer, self).__init__()

        self.conv1 = layers.Conv2D(kernels, (3, 3), padding='same')
        self.act1 = layers.Activation('relu')
        self.conv2 = layers.Conv2D(kernels, (1, 1), padding='same')
        self.act2 = layers.Activation('relu')
        
    def call(self, inputs):
        x = inputs
        y = self.conv1(x)
        y = self.act1(y)
        y = self.conv2(y)
        z = keras.layers.add([x, y])
        z = self.act2(z)

        return z
        

In [ ]:
class VQVAE(keras.models.Model):
    def __init__(self, train_variance, latent_dim=32, num_embeddings=128, **kwargs):
        super(VQVAE, self).__init__(**kwargs)
        self.train_variance = train_variance
        self.latent_dim = latent_dim
        self.num_embeddings = num_embeddings

        self.vqvae = self._get_vqvae(self.latent_dim, self.num_embeddings)

        self.total_loss_tracker = keras.metrics.Mean(name="loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.vq_loss_tracker = keras.metrics.Mean(name="vq_loss")

        
    def _get_vqvae(self, latent_dim=16, num_embeddings=64):
        vq_layer = VectorQuantizer(num_embeddings, latent_dim, name="vector_quantizer")
        self.encoder = self._get_encoder(latent_dim)
        self.decoder = self._get_decoder(input_shape=self.encoder.output.shape[1:])
        inputs = keras.Input(shape=(32, 32, 3))
        encoder_outputs = self.encoder(inputs)
        quantized_latents = vq_layer(encoder_outputs)
        reconstructions = self.decoder(quantized_latents)
        return keras.Model(inputs, reconstructions, name="vq_vae")
        
    def _get_encoder(self, latent_dim=16):
        encoder_inputs = keras.Input(shape=(32, 32, 3))
        x = layers.Conv2D(32, 4, activation="relu", strides=2, padding="same")(
            encoder_inputs
        )
        x = layers.Conv2D(64, 4, activation="relu", strides=2, padding="same")(x)
        x = ResidualLayer(64)(x)
        x = ResidualLayer(64)(x)
        encoder_outputs = layers.Conv2D(latent_dim, 1, padding="same")(x)
        return keras.Model(encoder_inputs, encoder_outputs, name="encoder")


    def _get_decoder(self, input_shape):
        latent_inputs = keras.Input(shape=input_shape)
        print(input_shape[-1])
        x = ResidualLayer(kernels=input_shape[-1])(latent_inputs)
        x = ResidualLayer(kernels=input_shape[-1])(x)
        x = layers.Conv2DTranspose(64, 4, activation="relu", strides=2, padding="same")(x)
        x = layers.Conv2DTranspose(32, 4, activation="relu", strides=2, padding="same")(x)
        decoder_outputs = layers.Conv2DTranspose(3, 4, padding="same")(x)
        return keras.Model(latent_inputs, decoder_outputs, name="decoder")
        
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.vq_loss_tracker,
        ]

    def train_step(self, x):
        with tf.GradientTape() as tape:
            # Outputs from the VQ-VAE.
            reconstructions = self.vqvae(x)

            # Calculate the losses.
            reconstruction_loss = (
                tf.reduce_mean(((x - reconstructions) ** 2)) #S/ self.train_variance
            )
            total_loss = reconstruction_loss + sum(self.vqvae.losses)

        # Backpropagation.
        grads = tape.gradient(total_loss, self.vqvae.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.vqvae.trainable_variables))

        # Loss tracking.
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.vq_loss_tracker.update_state(sum(self.vqvae.losses))

        # Log results.
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "vq_loss": self.vq_loss_tracker.result(),
        }
    
    
    def call(self, data):
        reconstructions = self.vqvae(data)
        
        # Calculate the losses.
        reconstruction_loss = (
            tf.reduce_mean(((data - reconstructions) ** 2)) #/ self.train_variance
        )
        total_loss = reconstruction_loss + sum(self.vqvae.losses)


        # actualizamos las métricas
        self.add_metric(total_loss, name='loss', aggregation='mean')
        self.add_metric(sum(self.vqvae.losses), name='vq_loss', aggregation='mean')
        self.add_metric(reconstruction_loss, name='reconstruction_loss', aggregation='mean')
        
        return reconstructions

In [ ]:
vqvae_trainer = VQVAE(data_variance, latent_dim=64, num_embeddings=512)
vqvae_trainer.encoder.summary()

In [ ]:
(x_train, _), (x_test, _) = keras.datasets.cifar10.load_data()

x_train_scaled = (x_train / 255.0) - 0.5
x_test_scaled = (x_test / 255.0) - 0.5


data_variance = np.var(x_train / 255.0)

In [ ]:
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, 
                                                          patience=5, verbose=1, min_lr=0.5e-6)   

vqvae_trainer = VQVAE(data_variance, latent_dim=64, num_embeddings=512)
vqvae_trainer.compile(optimizer=keras.optimizers.Adam(2e-4))
vqvae_trainer.fit(x_train_scaled, epochs=100, batch_size=128, validation_data=(x_test_scaled,),
                 callbacks=[PlotLossesKerasTF()])

In [ ]:
vqvae_trainer.fit(x_train_scaled, epochs=100, batch_size=128, validation_data=(x_test_scaled,),
                 callbacks=[PlotLossesKerasTF()])

In [ ]:
vqvae_trainer.fit(x_train_scaled, epochs=100, batch_size=128, validation_data=(x_test_scaled,),
                 callbacks=[PlotLossesKerasTF()])

In [ ]:
vqvae_trainer.fit(x_train_scaled, epochs=100, batch_size=128, validation_data=(x_test_scaled,),
                 callbacks=[PlotLossesKerasTF()])

In [ ]:
vqvae_trainer.fit(x_train_scaled, epochs=100, batch_size=128, validation_data=(x_test_scaled,),
                 callbacks=[PlotLossesKerasTF()])

In [ ]:
vqvae_trainer.fit(x_train_scaled, epochs=100, batch_size=128, validation_data=(x_test_scaled,),
                 callbacks=[PlotLossesKerasTF()])

In [ ]:
vqvae_trainer.fit(x_train_scaled, epochs=100, batch_size=128, validation_data=(x_test_scaled,),
                 callbacks=[PlotLossesKerasTF()])

In [ ]:
image_set = x_test_scaled
trained_vqvae_model = vqvae_trainer.vqvae
idx = np.random.choice(len(image_set), 10)
test_images = image_set[idx] 
reconstructions_test = trained_vqvae_model.predict(test_images)


draw2compare(test_images + 0.5, reconstructions_test + 0.5)

In [ ]:
encoder = vqvae_trainer.vqvae.get_layer("encoder")
quantizer = vqvae_trainer.vqvae.get_layer("vector_quantizer")

encoded_outputs = encoder.predict(test_images)
flat_enc_outputs = encoded_outputs.reshape(-1, encoded_outputs.shape[-1])
codebook_indices = quantizer.get_code_indices(flat_enc_outputs)
codebook_indices = codebook_indices.numpy().reshape(encoded_outputs.shape[:-1])


draw2compare(test_images + 0.5, codebook_indices) 
